<a href="https://colab.research.google.com/github/PietrH/common_wikidata_props/blob/main/get_wikidata_botanical_collector_properties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Counting the most common claims for an occupation on WikiData

In [1]:
# mount google drive to store data to
from google.colab import drive
drive.mount('/gdrive')



Mounted at /gdrive


In [2]:
!pip install sparqlwrapper
!pip install --upgrade plotly

# !pip install wikidata
# https://rdflib.github.io/sparqlwrapper/

     |████████████████████████████████| 482 kB 10.0 MB/s 
     |████████████████████████████████| 41 kB 672 kB/s 


In [3]:
import requests
import re
from collections import Counter
import itertools as it
import pandas as pd
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import plotly.express as px

In [7]:
#@title Number of objects to retreive
#@markdown Howmany objects should be queried from Wikidata? What is the limit?


limit = 200000  #@param {type: "number"}
#@markdown Select checkbox if only doing a quick test run:
quick_test_run = False  #@param {type: "boolean"}
#@markdown Select checkbox to remove reference claims from the count:
remove_references = True  #@param {type: "boolean"}

#@markdown ---

if(quick_test_run):
  limit = 300

In [8]:


endpoint_url = "https://query.wikidata.org/sparql"

# botanical collector
#query = """SELECT DISTINCT ?item ?itemLabel WHERE {
#  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
#  {
#    SELECT DISTINCT ?item ?property ?propertyLabel WHERE {
#      ?item p:P106 ?statement0.
#      ?statement0 (ps:P106/(wdt:P279*)) wd:Q2083925.
#      
#    }
#    LIMIT 100000
#  }
#}"""

# naturalist
#query = """SELECT DISTINCT ?item ?itemLabel WHERE {
#  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
#  {
#    SELECT DISTINCT ?item ?property ?propertyLabel WHERE {
#      ?item p:P106 ?statement0.
#      ?statement0 (ps:P106/(wdt:P279*)) wd:Q18805.
#      
#    }
#    LIMIT 100000
#  }
#}"""

# biologist
query = """SELECT DISTINCT ?item ?itemLabel WHERE {
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE]". }
  {
    SELECT DISTINCT ?item ?property ?propertyLabel WHERE {
      ?item p:P106 ?statement0.
      ?statement0 (ps:P106/(wdt:P279*)) wd:Q864503.
      
    }
    LIMIT %s
  }
}""" % (limit)


def get_results(endpoint_url, query):
    user_agent = "get-botcol-props Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

#for result in results["results"]["bindings"]:
#    print(result)

Number of entities found:

In [9]:
n_matches = len(results['results']['bindings'])
n_matches

143030

Let's use the wbgetentities api to get property information, 50 properties and all statements at a time (so one request per 50 properties)

In [10]:


def grouper(n,iterable):
  iterable = iter(iterable)
  return iter(lambda:list(it.islice(iterable,n)), [])

q_nums = []
for result in results['results']['bindings']:
  q_nums.append(result['itemLabel']['value'])

out = []


for q_num_group in grouper(50,q_nums):
  
  #q_num = result['itemLabel']['value']
  #reply = requests.get(f"https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&ids={q_num}")
  #json.loads(reply.json())
  reply = requests.get(f"https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&ids={'|'.join(q_num_group)}")
  out.append(re.findall('(?<=property\":\")P[0-9]+',reply.text))

c = Counter([item for sublist in out for item in sublist])
n_prop = len(set([item for sublist in out for item in sublist]))
#c.most_common(n_prop)
c.most_common(10)

[('P248', 548225),
 ('P813', 505249),
 ('P143', 310426),
 ('P106', 222593),
 ('P214', 196374),
 ('P854', 173820),
 ('P31', 143481),
 ('P1810', 138472),
 ('P586', 119287),
 ('P21', 114019)]

In [11]:
def get_datatype_json(json):
  return json["datatype"]

def detect_reference_prop(json):
  return str(json['claims']['P31']).find('Q18608359') > 0

def get_label_json(json):
  return json['labels']['en']['value']

df = pd.DataFrame(c.most_common(n_prop),columns=['Property','n'])

out = {}

for p_num_group in grouper(50,list(df.Property.values)):
  reply = requests.get(f"https://www.wikidata.org/w/api.php?action=wbgetentities&format=json&ids={'|'.join(p_num_group)}")
  out.update(reply.json()['entities'])

df['json'] = df['Property'].map(out)
df['Label'] = df['json'].map(get_label_json)
df['datatype'] = df['json'].map(get_datatype_json)
df['reference_property'] = df['json'].map(detect_reference_prop)





In [12]:
if(remove_references):
  df = df[~df['reference_property']]

In [13]:
%load_ext google.colab.data_table

In [14]:
df.drop('json', axis = 1)

,Property,n,Label,datatype,reference_property
3,P106,222593,occupation,wikibase-item,False
4,P214,196374,VIAF ID,external-id,False
6,P31,143481,instance of,wikibase-item,False
7,P1810,138472,named as,string,False
8,P586,119287,IPNI author ID,external-id,False
...,...,...,...,...,...
2243,P4319,1,Elite Prospects staff ID,external-id,False
2244,P3111,1,FEI athlete ID,external-id,False
2245,P8689,1,ITHL author ID,external-id,False
2246,P3926,1,USA Track & Field athlete ID,external-id,False


Some properties can occur more than once on an entity, especially claims regarding references; Because I think it's valuable to see what claims are added to references, I included these in the tally

In [15]:
%unload_ext google.colab.data_table

write output to csv and download


In [ ]:
#from os.path import exists
#from google.colab import files
#if(exists("/gdrive/MyDrive/biologist_properties_142901.csv")):
#  df = pd.read_csv("/gdrive/MyDrive/biologist_properties_142901.csv")
#else:
#  df.to_csv("/gdrive/MyDrive/biologist_properties_142901.csv")
#  files.download("/gdrive/MyDrive/biologist_properties_142901.csv")

In [17]:


fig = px.treemap(df, path=['datatype', 'Label'], values='n', title = "All claims for all Biologists and subclasses on Wikidata")
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [20]:
fig.write_html("biologist_142k_no-ref.html")

In [22]:
# Limit ourselves to statements that occur a certain amount
#df2 = df[df['n'] > 200]
df2 = df.head(50)

fig2 = px.treemap(df2, path=['datatype', 'Label'], values='n', title = "50 most common claims for all Biologists and subclasses on Wikidata")
fig2.update_traces(root_color="lightgrey")
fig2.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig2.show()

In [23]:
fig2.write_html("biologist_142k_no-ref_top50.html")

## Just claims and identifiers seperatly
Split to just claims and identifiers, show only top 10 of each. Mention that the occurrence is a required field because that's how we built the query.

In [28]:
df_simple = df[(df['datatype']!="external-id") & (df['datatype']!='Wikibase-item')]
df_simple


,Property,n,json,Label,datatype,reference_property
3,P106,222593,"{'pageid': 4164914, 'ns': 120, 'title': 'Prope...",occupation,wikibase-item,False
6,P31,143481,"{'pageid': 3918489, 'ns': 120, 'title': 'Prope...",instance of,wikibase-item,False
7,P1810,138472,"{'pageid': 21382977, 'ns': 120, 'title': 'Prop...",named as,string,False
9,P21,114019,"{'pageid': 3917971, 'ns': 120, 'title': 'Prope...",sex or gender,wikibase-item,False
10,P735,112166,"{'pageid': 16066975, 'ns': 120, 'title': 'Prop...",given name,wikibase-item,False
...,...,...,...,...,...,...
2215,P3618,1,"{'pageid': 30411741, 'ns': 120, 'title': 'Prop...",base salary,quantity,False
2219,P2975,1,"{'pageid': 27886155, 'ns': 120, 'title': 'Prop...",host,wikibase-item,False
2226,P7137,1,"{'pageid': 65722117, 'ns': 120, 'title': 'Prop...",acknowledged,wikibase-item,False
2241,P2360,1,"{'pageid': 23607049, 'ns': 120, 'title': 'Prop...",intended public,wikibase-item,False


In [46]:
no_ids = df[(df['datatype']!="external-id")]
fig5 = px.treemap(no_ids, path=['Label'], values='n',title = "Claims made about Biologists and subclasses of biologists on wikidata")
fig5.update_traces(root_color="lightgrey")
fig5.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig5.show()

In [52]:
fig5.write_html("biologist_142k_no-ref_claims.html")

In [50]:
no_ids_top15 = df[(df['datatype']!="external-id")].head(15)
fig6 = px.treemap(no_ids_top15, path=['Label'], values='n',title = "15 most common claims made about Biologists and subclasses of biologists on wikidata")
fig6.update_traces(root_color="lightgrey")
fig6.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig6.show()

In [51]:
fig6.write_html("biologist_142k_no-ref_claims_top15.html")

In [56]:
no_ids.drop(['json','datatype','reference_property'], axis = 1).to_csv("biologist_142k_claims_tally.csv")

#no_ids.head(1000).to_csv("biologist_142k_claims_tally_top1000.csv")

## Identifiers only

In [40]:
# just identifiers

only_identifiers = df[(df['datatype']=="external-id")].drop(['json','datatype','reference_property'], axis = 1)
only_identifiers.to_csv("biologist_142k_identifiers_tally.csv")
only_identifiers

,Property,n,Label
4,P214,196374,VIAF ID
8,P586,119287,IPNI author ID
13,P428,70060,botanist author abbreviation
18,P227,45784,GND ID
19,P7859,45567,WorldCat Identities ID
...,...,...,...
2242,P5666,1,Quais du polar writer ID
2243,P4319,1,Elite Prospects staff ID
2244,P3111,1,FEI athlete ID
2245,P8689,1,ITHL author ID


Place all the claims except external id's in the same category, show the most common ones (10) in a treemap. 

In [38]:
fig3 = px.treemap(only_identifiers,path=['Label'],values = 'n',title = 'Most commonly used identifiers for Biologists and subclasses of biologists on Wikidata')
fig3.update_traces(root_color="lightgrey")
fig3.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig3.show()

In [39]:
fig3.write_html("biologist_142k_no-ref_only_id.html")

In [44]:
identifiers_top_20 = df[(df['datatype']=="external-id")].drop(['json','datatype','reference_property'], axis = 1).head(20)
fig4 = px.treemap(identifiers_top_20,path=['Label'],values = 'n',title = 'The 20 most common identifiers for Biologists and subclasses of biologists on Wikidata', width = 800, height = 800)
fig4.update_traces(root_color="lightgrey")
fig4.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig4.show()

In [45]:
fig4.write_html("biologist_142k_top20_ids.html")

## Identifiers only

A closer look at the external identifiers only, a treemap and a table with all of the identifiers sorted by how often they are used. Also one with just the top 20.



